In [3]:
import json

def normalize(value, min_val, max_val):
    """주어진 값을 0과 1 사이의 값으로 정규화하는 헬퍼 함수."""
    # 응답이 없는 경우(None)는 그대로 반환
    if value is None:
        return None
    # (현재값 - 최소값) / (최대값 - 최소값) 공식을 적용
    return round((value - min_val) / (max_val - min_val), 4)

def calculate_user_profile_normalized(survey_json_string):
    """
    사용자의 설문 응답 JSON을 받아, 수치적 특성을 0과 1 사이로 정규화하여
    분석된 특성 프로필 딕셔너리를 반환하는 메인 알고리즘 함수입니다.
    """
    try:
        responses = json.loads(survey_json_string)
    except json.JSONDecodeError:
        print("오류: 유효하지 않은 JSON 형식의 입력입니다.")
        return None

    user_profile = extract_and_normalize_features(responses)
    return user_profile

def extract_and_normalize_features(responses):
    """
    설문 응답 딕셔너리를 받아 사용자 특성을 추출하고,
    수치 데이터를 0과 1 사이로 정규화하여 반환합니다.
    """
    features = {
        'FSC': {}, 'PSSR': {}, 'MP': {}, 'DLS': {}
    }

    # --- 기능적 및 구조적 역량 (FSC) ---
    # 각 문항의 최소/최대값을 기준으로 0~1 정규화
    features['FSC']['time_availability'] = normalize(responses.get('1'), 1, 4)
    features['FSC']['financial_budget'] = normalize(responses.get('2'), 1, 4)
    features['FSC']['energy_level'] = normalize(responses.get('3'), 1, 5)
    features['FSC']['mobility'] = normalize(responses.get('4'), 1, 5)
    
    # 불리언 값은 그대로 유지
    features['FSC']['has_physical_constraints'] = True if responses.get('5') in [1, 2, 3] else False
    features['FSC']['has_housing_constraints'] = True if responses.get('12') in [2, 3, 4] else False
    
    # 텍스트 값은 그대로 유지
    features['FSC']['preferred_space'] = 'indoor' if responses.get('6') == 1 else 'outdoor'

    # --- 심리적 안전 및 사회적 준비도 (PSSR) ---
    # 점수를 계산한 후, 해당 점수 범위(1~5)를 기준으로 0~1 정규화
    q13 = responses.get('13', 3); q14_r = 6 - responses.get('14', 3); q16 = responses.get('16', 3)
    self_criticism_raw = (q13 + q14_r + q16) / 3
    features['PSSR']['self_criticism_score'] = normalize(self_criticism_raw, 1, 5)

    q15 = responses.get('15', 3); q18 = responses.get('18', 3); q20 = responses.get('20', 3)
    social_anxiety_raw = (q15 + q18 + q20) / 3
    features['PSSR']['social_anxiety_score'] = normalize(social_anxiety_raw, 1, 5)
    
    features['PSSR']['isolation_level'] = normalize(responses.get('21'), 1, 5)
    features['PSSR']['structure_preference_score'] = normalize(responses.get('27'), 1, 5)
    features['PSSR']['avoidant_coping_score'] = normalize(responses.get('29'), 1, 5)

    # --- 동기 프로필 (MP) ---
    # 텍스트 값은 그대로 유지
    motivation_map = {1: 'achievement', 2: 'recovery', 3: 'connection', 4: 'vitality'}
    features['MP']['core_motivation'] = motivation_map.get(responses.get('31'))
    
    # 딕셔너리 내부의 모든 수치 값을 정규화
    features['MP']['value_profile'] = {
        'knowledge': normalize(responses.get('33'), 1, 5),
        'stability': normalize(responses.get('34'), 1, 5),
        'relationship': normalize(responses.get('35'), 1, 5),
        'health': normalize(responses.get('36'), 1, 5),
        'creativity': normalize(responses.get('37'), 1, 5),
        'control': normalize(responses.get('38'), 1, 5),
    }
    features['MP']['process_orientation_score'] = normalize(6 - responses.get('41', 3), 1, 5)

    # --- 선호하는 사회성 수준 (DLS) ---
    # 텍스트 값은 그대로 유지
    sociality_map = {1: 'solo', 2: 'parallel', 3: 'low_interaction_group', 4: 'high_interaction_group'}
    features['DLS']['preferred_sociality_type'] = sociality_map.get(responses.get('39'))
    group_size_map = {1: 'one_on_one', 2: 'small_group', 3: 'large_group'}
    features['DLS']['preferred_group_size'] = group_size_map.get(responses.get('40'))
    
    features['DLS']['autonomy_preference_score'] = normalize(responses.get('42'), 1, 5)

    return features

In [4]:
if __name__ == '__main__':
    user_survey_json = """
    {
        "1": 1, "2": 1, "3": 2, "4": 2, "5": 3, "6": 1, "12": 2,
        "13": 4, "14": 4, "15": 5, "16": 4, "18": 5, "20": 4,
        "21": 1, "27": 1, "29": 2,
        "31": 2, "33": 3, "34": 5, "35": 1, "36": 3, "37": 5, "38": 4,
        "39": 1, "40": 1, "41": 4, "42": 1
    }
    """
    
    print("--- 1. 사용자 설문 응답 (JSON) ---")
    print(user_survey_json)

    print("\n--- 2. 정규화된 프로필 계산 실행 ---")
    calculated_profile = calculate_user_profile_normalized(user_survey_json)
    
    print("\n--- 3. 최종 계산된 사용자 프로필 (0-1 정규화) ---")
    if calculated_profile:
        print(json.dumps(calculated_profile, indent=4, ensure_ascii=False))

--- 1. 사용자 설문 응답 (JSON) ---

    {
        "1": 1, "2": 1, "3": 2, "4": 2, "5": 3, "6": 1, "12": 2,
        "13": 4, "14": 4, "15": 5, "16": 4, "18": 5, "20": 4,
        "21": 1, "27": 1, "29": 2,
        "31": 2, "33": 3, "34": 5, "35": 1, "36": 3, "37": 5, "38": 4,
        "39": 1, "40": 1, "41": 4, "42": 1
    }
    

--- 2. 정규화된 프로필 계산 실행 ---

--- 3. 최종 계산된 사용자 프로필 (0-1 정규화) ---
{
    "FSC": {
        "time_availability": 0.0,
        "financial_budget": 0.0,
        "energy_level": 0.25,
        "mobility": 0.25,
        "has_physical_constraints": true,
        "has_housing_constraints": true,
        "preferred_space": "indoor"
    },
    "PSSR": {
        "self_criticism_score": 0.5833,
        "social_anxiety_score": 0.9167,
        "isolation_level": 0.0,
        "structure_preference_score": 0.0,
        "avoidant_coping_score": 0.25
    },
    "MP": {
        "core_motivation": "recovery",
        "value_profile": {
            "knowledge": 0.5,
            "stability": 1.0